# Install libs

In [ ]:
%pip install pymongo openpyxl python-dotenv

# Imports + config

In [5]:
import os
from collections import OrderedDict
from datetime import datetime, date
from decimal import Decimal

from dotenv import load_dotenv
from openpyxl import load_workbook
from pymongo import MongoClient

EXCEL_PATH = r"../wine-cart/public/wine_data.xlsx"   # adjust if needed
DB_NAME = "wine_cart"
COLLECTION_NAME = "wines"
SHEET_NAME = None    # e.g. "Sheet1", or keep None for active sheet
DROP_FIRST = True    # set False if you don't want to clear collection

# Helper function

In [6]:
def normalize_value(v):
    if isinstance(v, (datetime, date)):
        return v.isoformat()
    if isinstance(v, Decimal):
        return float(v)
    return v

# Load Mongo URI and connect

In [7]:
load_dotenv("../wine-cart/.env")  # adjust path if your .env is elsewhere
mongo_uri = os.getenv("MONGODB_URI")

if not mongo_uri:
    raise ValueError("MONGODB_URI not found. Put it in your .env file.")

client = MongoClient(mongo_uri)
collection = client[DB_NAME][COLLECTION_NAME]
print("Connected to MongoDB")

Connected to MongoDB


# Read Excel and preserve column order

In [8]:
wb = load_workbook(EXCEL_PATH, data_only=True)
ws = wb[SHEET_NAME] if SHEET_NAME else wb.active

rows = ws.iter_rows(values_only=True)
headers = next(rows)

# preserve exact left-to-right column order from Excel
headers = [str(h).strip() if h is not None else f"col_{i}" for i, h in enumerate(headers)]

docs = []
for row in rows:
    if row is None or all(c is None for c in row):
        continue
    doc = OrderedDict()
    for h, c in zip(headers, row):
        doc[h] = normalize_value(c)
    docs.append(doc)

print(f"Prepared {len(docs)} documents")
print("Column order:", headers)

Prepared 726 documents
Column order: ['wine_id', 'appellation', 'producer', 'region', 'country', 'vintage', 'wine_type', 'grape_variety', 'alcohol_content', 'volume', 'image_filename', 'price', 'description']


# Insert into MongoDB

In [9]:
if DROP_FIRST:
    collection.drop()
    print(f"Dropped {DB_NAME}.{COLLECTION_NAME}")

if docs:
    result = collection.insert_many(docs, ordered=True)
    print(f"Inserted {len(result.inserted_ids)} docs into {DB_NAME}.{COLLECTION_NAME}")
else:
    print("No data rows found")

Dropped wine_cart.wines
Inserted 726 docs into wine_cart.wines


# Quick verify first doc key order

In [10]:
first_doc = collection.find_one({}, {"_id": 0})
print(list(first_doc.keys()))

['wine_id', 'appellation', 'producer', 'region', 'country', 'vintage', 'wine_type', 'grape_variety', 'alcohol_content', 'volume', 'image_filename', 'price', 'description']
